# Memory

## Intro
* Ability to store information about past interactions.
* **Most of memory-related functionality in LangChain is marked as beta**. This is for two reasons:
    1. Most functionality are not production ready.
    2. Most functionality work with Legacy chains, not the newer LCEL syntax.
* **The main exception to this is the ChatMessageHistory functionality**. This functionality is largely production ready and does integrate with LCEL.

## LangChain documentation on Chains
* See the LangChain documentation page on Memory [here](https://python.langchain.com/v0.1/docs/modules/memory/).
* See the LangChain documentation page on how to use ChatMessageHistory with LCEL [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).
* See the LangChain documentation page on the various ChatMessageHistory integrations [here](https://python.langchain.com/v0.1/docs/integrations/memory/).

## Setup

#### Recommended: create new virtualenv
* mkdir your_project_name
* cd your_project_name
* pyenv virtualenv 3.11.4 your_venv_name
* pyenv activate your_venv_name
* pip install jupyterlab
* jupyter lab

In [1]:
#!pip install python-dotenv

#### .env File
Remember to include:
OPENAI_API_KEY=your_openai_api_key

LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=your_langchain_api_key
LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **memory**.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

In [3]:
#!pip install langchain

## Connect with an LLM

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

## Buffer Memory
* ConversationBufferMemory keeps a list of chat messages in a buffer and passes those into the prompt template.
* A buffer refers to a temporary storage area in memory used to hold data. Here, the buffer specifically holds chat messages before they are processed or used in some way, such as being passed into a prompt template.
* In simple terms, a buffer is like a waiting room for data. It's a temporary holding spot where data can stay until it's ready to be used or processed. Imagine you're cooking and you chop up vegetables before you need to cook them. Instead of chopping each vegetable right before it goes in the pan, you chop them all at once and put them aside on a cutting board. That cutting board with the chopped vegetables is like a buffer — it holds everything ready for you until you're ready to use it. This way, when you need the vegetables, they are all prepared and ready to go, making the cooking process smoother and faster.

In [3]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv060824-p2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [4]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv060824-p2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?')],
 'text': 'Hello! How are you today?'}

## ChatMessageHistory
* ChatMessageHistory provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

In [ ]:
#!pip install langchain-community

In [6]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [7]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]